In [1]:
### 라이브러리
import pandas as pd
import numpy as np
import os
import json
import cv2
import matplotlib.pyplot as plt
import plotly.graph_objects as ply
import tensorflow as tf
from tensorflow import keras
### 교차검증
from sklearn.model_selection import cross_validate

### 교차검증 -> KFold 라이브러리
from sklearn.model_selection import StratifiedKFold

### AutoML(그리드서치) 라이브러리
from sklearn.model_selection import GridSearchCV

### AutoML(랜덤서치) 라이브러리
from sklearn.model_selection import RandomizedSearchCV
### AutoML(랜덤서치)에서 사용할 확률분포 데이터 생성라이브러리
from scipy.stats import randint, uniform

# 훈련, 테스트 파일 불러오기

In [259]:
### 제이슨 파일 불러오기
directory_path = './data/라벨링/sp_1'
json_data = []

try:
    file_list = os.listdir(directory_path)
    json_files = [file for file in file_list if file.endswith('.json')]

    for json_file in json_files:
        json_path = os.path.join(directory_path, json_file)
        try:
            with open(json_path, 'r') as file:
                data = json.load(file)
                json_data.append(data)
        except Exception as e:
            print(f"Error reading JSON file '{json_file}': {e}")
except Exception as e:
    print(f"Error accessing directory '{directory_path}': {e}")

In [66]:
### 제이슨 파일 불러오기
directory_path = './자료/라벨링/sp_2'

try:
    file_list = os.listdir(directory_path)
    json_files = [file for file in file_list if file.endswith('.json')]

    for json_file in json_files:
        json_path = os.path.join(directory_path, json_file)
        try:
            with open(json_path, 'r') as file:
                data = json.load(file)
                json_data.append(data)
        except Exception as e:
            print(f"Error reading JSON file '{json_file}': {e}")
except Exception as e:
    print(f"Error accessing directory '{directory_path}': {e}")

In [77]:
### 제이슨 파일 불러오기

for i in range(5,30,1):
    directory_path = './자료/라벨링/sp_{}'.format(i)
    

    try:
        file_list = os.listdir(directory_path)
        json_files = [file for file in file_list if file.endswith('.json')]

        for json_file in json_files:
            json_path = os.path.join(directory_path, json_file)
            try:
                with open(json_path, 'r') as file:
                    data = json.load(file)
                    json_data.append(data)
            except Exception as e:
                print(f"Error reading JSON file '{json_file}': {e}")
    except Exception as e:
        print(f"Error accessing directory '{directory_path}': {e}")

Error accessing directory './자료/라벨링/sp_5': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자료/라벨링/sp_5'
Error accessing directory './자료/라벨링/sp_6': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자료/라벨링/sp_6'
Error accessing directory './자료/라벨링/sp_7': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자료/라벨링/sp_7'
Error accessing directory './자료/라벨링/sp_8': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자료/라벨링/sp_8'
Error accessing directory './자료/라벨링/sp_9': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자료/라벨링/sp_9'
Error accessing directory './자료/라벨링/sp_10': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자료/라벨링/sp_10'
Error accessing directory './자료/라벨링/sp_11': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자료/라벨링/sp_11'
Error accessing directory './자료/라벨링/sp_12': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자료/라벨링/sp_12'
Error accessing directory './자료/라벨링/sp_13': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자료/라벨링/sp_13'
Error accessing directory './자료/라벨링/sp_14': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자료/라벨링/sp_14'
Error accessing directory './자료/라벨링/sp_15': [WinError 3] 지정된 경로를 찾을 수 없습니다: './자

### 예측 데이터 생성

In [225]:
### 예측을 위한 제이슨 파일 불러오기
json_data = []
for i in range(1,13,1):
    directory_path = './data/smart/smart({})'.format(i)
    
    try:
        file_list = os.listdir(directory_path)
        json_files = [file for file in file_list if file.endswith('.json')]

        for json_file in json_files:
            json_path = os.path.join(directory_path, json_file)
            try:
                with open(json_path, 'r') as file:
                    data = json.load(file)
                    json_data.append(data)
            except Exception as e:
                print(f"Error reading JSON file '{json_file}': {e}")
    except Exception as e:
        print(f"Error accessing directory '{directory_path}': {e}")
        
### 제이슨 파일 데이터프레임으로 변환
df = pd.DataFrame()

for data in json_data:
    ### ID
    id_ = data['Annotations']['metaid']
    ### point
    point = data['Annotations']['pose']['point']
    ### condition
    condition = data['Annotations']['condition']
    
    df_1 = pd.DataFrame({'ID':id_, 'point':[point], 'condition':condition})
    
    ### label 별로 평균값으로 데이터 프레임 넣기    
    for item in data['Annotations']['annotations']:
        label = item['label']
        points = item['points']
        avg_x = sum(p[0] for p in points) / len(points)
        avg_y = sum(p[1] for p in points) / len(points)

        df_1[label+'_x'] = avg_x
        df_1[label+'_y'] = avg_y
    
    ### df에 추가하기
    df = pd.concat ([df, df_1], ignore_index = True)

In [226]:
### 결측치 처리
df = df.dropna()

### 독립, 종속변수 분리
data = df.iloc[:, 3:].to_numpy()
target_pred = df.iloc[:, 2].to_numpy()
print(data.shape, target_pred.shape)
### target 데이터를 숫자로 인코딩
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encoder = LabelEncoder()

# 데이터 변환
target_pred = label_encoder.fit_transform(target_pred)

print(target_pred.shape)

### StandardScaler
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(data)

pred_std_scaler = ss.transform(data)

print(pred_std_scaler.shape)

(48465, 12) (48465,)
(48465,)
(48465, 12)


In [252]:
df

,ID,point,condition,r_iris_x,r_iris_y,l_iris_x,l_iris_y,r_eyelid_x,r_eyelid_y,l_eyelid_x,l_eyelid_y,l_center_x,l_center_y,r_center_x,r_center_y
0,001,"[1020, 782]",A,987.5,362.0,816.0,386.5,991.6875,365.2500,818.0625,390.5000,816.0,386.5,987.5,362.0
1,001,"[1020, 782]",A,989.0,363.0,818.0,388.0,993.1875,366.8750,818.9375,392.6875,818.0,388.0,989.0,363.0
2,001,"[1020, 782]",A,989.5,362.5,819.0,388.0,993.0000,366.9375,819.5625,392.4375,819.0,388.0,989.5,362.5
3,001,"[1020, 782]",A,988.5,363.0,817.0,388.0,990.5625,366.3750,819.3750,391.2500,817.0,388.0,988.5,363.0
4,001,"[1020, 782]",A,986.0,362.0,814.5,387.0,990.4375,365.3125,815.3125,390.7500,814.5,387.0,986.0,362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,001,"[923, 499]",S,652.0,767.5,382.5,761.0,652.1250,765.6875,384.4375,757.0000,383.0,761.0,654.0,769.0
3212,001,"[923, 499]",S,654.0,768.5,389.0,757.5,655.5625,765.3750,380.9375,755.0000,389.0,757.5,654.0,768.5
3213,001,"[923, 499]",S,651.5,769.0,383.0,759.5,653.5000,764.3750,378.9375,754.1250,389.0,753.5,658.0,767.0
3214,001,"[923, 499]",S,655.5,768.5,390.5,755.0,650.1875,765.0625,382.7500,755.0625,390.5,755.0,655.5,768.5


In [141]:
### 예측하기
pred = et.predict(pred_std_scaler)
tr = sum(pred==target)
fs = sum(pred!=target)
# - 결과출력
print("정답갯수[{}] / 오답갯수[{}] / 정확도[{}]% / 오답률[{}]%".format(tr, fs, round(tr/(tr+fs)*100,1),round(fs/(tr+fs)*100,1)))

정답갯수[9728] / 오답갯수[38737] / 정확도[20.1]% / 오답률[79.9]%


In [199]:
### 예측하기
pred = et.predict(test_std_scaler)
tr = sum(pred==test_y)
fs = sum(pred!=test_y)
# - 결과출력
print("정답갯수 {}개 / 오답갯수 {}개 / 정확도 {}% / 오답률 {}%".format(tr, fs, round(tr/(tr+fs)*100,1),round(fs/(tr+fs)*100,1)))

정답갯수 42591개 / 오답갯수 3953개 / 정확도 91.5% / 오답률 8.5%


In [227]:
### 예측하기
pred = rf.predict(pred_std_scaler)
tr = sum(pred==target_pred)
fs = sum(pred!=target_pred)
# - 결과출력
print("정답갯수 {}개 / 오답갯수 {}개 / 정확도 {}% / 오답률 {}%".format(tr, fs, round(tr/(tr+fs)*100,1),round(fs/(tr+fs)*100,1)))

정답갯수 9468개 / 오답갯수 38997개 / 정확도 19.5% / 오답률 80.5%


In [11]:
### 제이슨 파일 불러오기
directory_path = './data/라벨링/sp_1'
json_data = []

try:
    file_list = os.listdir(directory_path)
    json_files = [file for file in file_list if file.endswith('.json')]

    for json_file in json_files:
        json_path = os.path.join(directory_path, json_file)
        try:
            with open(json_path, 'r') as file:
                data = json.load(file)
                json_data.append(data)
        except Exception as e:
            print(f"Error reading JSON file '{json_file}': {e}")
except Exception as e:
    print(f"Error accessing directory '{directory_path}': {e}")

### 제이슨 파일 데이터프레임으로 변환
df = pd.DataFrame()

for data in json_data:
    ### ID
    id_ = data['Annotations']['metaid']
    ### point
    point = data['Annotations']['pose']['point']
    ### condition
    condition = data['Annotations']['condition']
    
    df_1 = pd.DataFrame({'ID':id_, 'point':[point], 'condition':condition})
    
    ### label 별로 평균값으로 데이터 프레임 넣기    
    for item in data['Annotations']['annotations']:
        label = item['label']
        points = item['points']
        avg_x = sum(p[0] for p in points) / len(points)
        avg_y = sum(p[1] for p in points) / len(points)

        df_1[label+'_x'] = avg_x
        df_1[label+'_y'] = avg_y
    
    ### df에 추가하기
    df = pd.concat ([df, df_1], ignore_index = True)
df

,ID,point,condition,r_iris_x,r_iris_y,l_iris_x,l_iris_y,r_eyelid_x,r_eyelid_y,l_eyelid_x,l_eyelid_y,l_center_x,l_center_y,r_center_x,r_center_y
0,001,"[1020, 782]",A,987.5,362.0,816.0,386.5,991.6875,365.2500,818.0625,390.5000,816.0,386.5,987.5,362.0
1,001,"[1020, 782]",A,989.0,363.0,818.0,388.0,993.1875,366.8750,818.9375,392.6875,818.0,388.0,989.0,363.0
2,001,"[1020, 782]",A,989.5,362.5,819.0,388.0,993.0000,366.9375,819.5625,392.4375,819.0,388.0,989.5,362.5
3,001,"[1020, 782]",A,988.5,363.0,817.0,388.0,990.5625,366.3750,819.3750,391.2500,817.0,388.0,988.5,363.0
4,001,"[1020, 782]",A,986.0,362.0,814.5,387.0,990.4375,365.3125,815.3125,390.7500,814.5,387.0,986.0,362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,001,"[923, 499]",S,652.0,767.5,382.5,761.0,652.1250,765.6875,384.4375,757.0000,383.0,761.0,654.0,769.0
3212,001,"[923, 499]",S,654.0,768.5,389.0,757.5,655.5625,765.3750,380.9375,755.0000,389.0,757.5,654.0,768.5
3213,001,"[923, 499]",S,651.5,769.0,383.0,759.5,653.5000,764.3750,378.9375,754.1250,389.0,753.5,658.0,767.0
3214,001,"[923, 499]",S,655.5,768.5,390.5,755.0,650.1875,765.0625,382.7500,755.0625,390.5,755.0,655.5,768.5


In [38]:
# 이미지 파일이 있는 폴더 경로
image_folder = './data/model/'

# 이미지 파일들의 경로 가져오기
image_paths = glob.glob(image_folder + '*.png')

# 데이터프레임 순회하며 결측치가 있는 행 삭제하고 이미지 파일도 삭제
rows_to_delete = []
for index, row in df.iterrows():
    if pd.isnull(row).any():  # 결측치 있는 행
        rows_to_delete.append(index)

# 데이터프레임에서 결측치가 있는 행 삭제
df.drop(rows_to_delete, inplace=True)

# 해당 행에 해당하는 이미지 파일도 리스트에서 삭제
for index in rows_to_delete:
    image_path = image_paths[index]
    os.remove(image_path)

# 이미지 파일이 있는 폴더 경로
image_folder = './data/model/'

# 이미지 파일들의 경로 가져오기
image_paths = glob.glob(image_folder + '*.png')

In [39]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import glob



features = []
r=0
# hog 함수를 이용하여 이미지 파일 읽어들이기
for image_file in image_paths:
#     print(r)
#     r=r+1
    image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    hog_features = hog(image, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")
    features.append(hog_features)

# numpy 배열을 DataFrame으로 변환 후, 원래의 DataFrame에 추가
df_hog = pd.DataFrame(features)
df = pd.concat([df, df_hog], axis=1)
df

MemoryError: Unable to allocate 15.8 MiB for an array with shape (1920, 1080) and data type float64

In [37]:
image = cv2.imread(image_paths[176], cv2.IMREAD_GRAYSCALE)
hog_features = hog(image, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")
hog_features
image_paths[174]

'./data/model\\NIA22EYE_S1_001_T1_S10_S_rgb_A_E_L_109.png'

In [321]:
# y = []
# label_encoder = LabelEncoder()
# labels = df.iloc[:,2]  # 라벨 컬럼 선택
# encoded_labels = label_encoder.fit_transform(labels)
# for i in encoded_labels :
#     y.append(i)
# # HOG 특성과 레이블 설정
# X = []


# for image_path, coordinates in zip(image_paths, df.iloc[:,3:]):
#     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#     hog_features, _ = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
#     X.append(hog_features)  # 1차원으로 변환


# X = np.array(X)  # 2차원 배열로 변환
# y = np.array(y)

# X.shape,y.shape
df.reset_index(drop=True)

X = []
y = []
r=0
for image_path, coordinates, label in zip(image_paths, df.iloc[:, 3:], df.iloc[:, 2]):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    hog_features, _ = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    X.append(hog_features)  # 1차원으로 변환
    y.append(label)  # 레이블 추가

X = np.array(X)  # 2차원 배열로 변환
y = np.array(y)

X.shape, y.shape

0
1
2
3
4
5
6
7
8
9
10
11


((12, 1152936), (12,))

In [322]:
image

array([[ 22,  22,  15, ..., 141, 142, 143],
       [ 26,  22,  26, ..., 141, 141, 139],
       [ 31,  31,  31, ..., 140, 139, 137],
       ...,
       [168, 166, 164, ..., 138, 140, 141],
       [161, 165, 164, ..., 145, 146, 146],
       [164, 165, 166, ..., 146, 146, 146]], dtype=uint8)

In [312]:
# 훈련 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SVM 모델 생성 및 훈련
model = SVC(kernel='linear', random_state=42)
model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

ValueError: The number of classes has to be greater than one; got 1 class

In [260]:
### 제이슨 파일 데이터프레임으로 변환
df = pd.DataFrame()

for data in json_data:
    ### ID
    id_ = data['Annotations']['metaid']
    ### point
    point = data['Annotations']['pose']['point']
    ### condition
    condition = data['Annotations']['condition']
    
    df_1 = pd.DataFrame({'ID':id_, 'point':[point], 'condition':condition})
    
    ### label 별로 평균값으로 데이터 프레임 넣기    
    for item in data['Annotations']['annotations']:
        label = item['label']
        points = item['points']
        avg_x = sum(p[0] for p in points) / len(points)
        avg_y = sum(p[1] for p in points) / len(points)

        df_1[label+'_x'] = avg_x
        df_1[label+'_y'] = avg_y
    
    ### df에 추가하기
    df = pd.concat ([df, df_1], ignore_index = True)

In [80]:
### 결측치 처리
df = df.dropna()

In [81]:
data = df.iloc[:, 3:].to_numpy()
target = df.iloc[:, 2].to_numpy()

print(data.shape, target.shape)

(29339, 12) (29339,)


In [82]:
### target 데이터를 숫자로 인코딩
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encoder = LabelEncoder()

# 데이터 변환
target = label_encoder.fit_transform(target)

print(target.shape)

(29339,)


In [83]:
### 8:2로 나누기
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(data, target, random_state=42, test_size=0.2)

print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

(23471, 12) (23471,)
(5868, 12) (5868,)


### 정규화

In [84]:
### StandardScaler
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_x)

train_std_scaler = ss.transform(train_x)
test_std_scaler = ss.transform(test_x)

print(train_std_scaler.shape)

(23471, 12)


In [85]:
### 예측 데이터 정규화
### StandardScaler
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(data)

pred_std_scaler = ss.transform(data)

print(pred_std_scaler.shape)

(29339, 12)


### 랜덤포레스트

In [74]:
### 랜덤포레스트 훈련모델 라이브러리
from sklearn.ensemble import RandomForestClassifier

### 훈련모델 생성
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

### Std훈련 시키기
rf.fit(train_std_scaler, train_y)

### 훈련정확도 및 테스트 정확도
tr_score = rf.score(train_std_scaler, train_y)
ts_score = rf.score(test_std_scaler, test_y)

print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

훈련 정확도 : 0.9999843808571786
테스트 정확도 : 0.9159117885924908


In [222]:
### 랜덤포레스트 훈련모델 라이브러리
from sklearn.ensemble import RandomForestClassifier

### 훈련모델 생성
rf = RandomForestClassifier(n_jobs=-1,max_depth=39,n_estimators=100, random_state=42)

### Std훈련 시키기
rf.fit(train_std_scaler, train_y)

### 훈련정확도 및 테스트 정확도
tr_score = rf.score(train_std_scaler, train_y)
ts_score = rf.score(test_std_scaler, test_y)

print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

훈련 정확도 : 0.9999946287095475
테스트 정확도 : 0.9025653145410794


In [75]:
### 랜덤포레스트 교차검증
from sklearn.ensemble import RandomForestClassifier

params = {"max_depth" : range(5,20,1),
          "n_estimators" : range(2,30,1)}

### 훈련모델 생성
rf = RandomForestClassifier(random_state=42)

### AutoML 클래스 생성하기
gs = GridSearchCV(rf, params, n_jobs=-1)

### AutoML 실행하기 : 하이퍼파라메터 튜닝 + 훈련(교차검증 수행)
gs.fit(train_std_scaler, train_y)

rf = gs.best_estimator_

### 과적합 확인하기
print(gs.best_params_)
print("훈련 정확도 :", rf.score(train_std_scaler, train_y))
print("테스트 정확도 :", rf.score(test_std_scaler, test_y))

{'max_depth': 19, 'n_estimators': 26}
훈련 정확도 : 0.8925559165313007
테스트 정확도 : 0.8035859311551197


### 엑스트라 모델

In [28]:
from sklearn.ensemble import ExtraTreesClassifier

In [76]:
### 모델 생성
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

### 훈련시키기
et.fit(train_std_scaler, train_y)

### 정확도 확인
tr_score = et.score(train_std_scaler, train_y)
ts_score = et.score(test_std_scaler, test_y)
print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

훈련 정확도 : 1.0
테스트 정확도 : 0.9310301742987444


In [97]:
### 엑스트라 교차학습

params = {"max_depth" : range(1,25,1),
          "n_estimators" : range(2,40,1)}

### 훈련모델 생성
et = ExtraTreesClassifier(random_state=42)

### AutoML 클래스 생성하기
gs = GridSearchCV(et, params, n_jobs=-1)

### AutoML 실행하기 : 하이퍼파라메터 튜닝 + 훈련(교차검증 수행)
gs.fit(train_std_scaler, train_y)

et = gs.best_estimator_

### 과적합 확인하기
print(gs.best_params_)
print("훈련 정확도 :", et.score(train_std_scaler, train_y))
print("테스트 정확도 :", et.score(test_std_scaler, test_y))

{'max_depth': 24, 'n_estimators': 37}
훈련 정확도 : 0.9967668374359615
테스트 정확도 : 0.9119135378272006


### 그레디언트 부스팅

In [31]:
from sklearn.ensemble import GradientBoostingClassifier

In [78]:
### 모델 생성
gb = GradientBoostingClassifier(random_state=42)

### 훈련
gb.fit(train_std_scaler, train_y)

### 정확도
tr_score = gb.score(train_std_scaler, train_y)
ts_score = gb.score(test_std_scaler, test_y)
print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

훈련 정확도 : 0.4043015119330251
테스트 정확도 : 0.37870931467482977


### Histogram-base Gradient Boosting

In [33]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [79]:
### 모델 생성
hgb = HistGradientBoostingClassifier(random_state=42, )

### 훈련
hgb.fit(train_std_scaler, train_y)

### 정확도
tr_score = hgb.score(train_std_scaler, train_y)
ts_score = hgb.score(test_std_scaler, test_y)
print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

훈련 정확도 : 0.741518805447957
테스트 정확도 : 0.6919472730680327


### 결정트리

In [96]:
from sklearn.tree import DecisionTreeClassifier
# 모델생성
dt = DecisionTreeClassifier(max_depth=42, random_state=42)
# 훈련시키기
dt.fit(train_std_scaler, train_y)
tr_score = dt.score(train_std_scaler, train_y)
ts_score = dt.score(test_std_scaler, test_y)
print("훈련데이터 정확도 :", tr_score,
      "\n테스트데이터 정확도 :", ts_score)

훈련데이터 정확도 : 0.9999843808571786 
테스트데이터 정확도 : 0.836321609295933


# 통합파일 처리

In [106]:
### 파일 저장
df.to_csv("./자료/smartphone_label.csv",index=False)

In [122]:
### 파일 불러오기
df = pd.read_csv("./data/smartphone_label.csv")

In [123]:
### 통합 준비
df_smart = df.copy()

In [124]:
### laptop파일 불러오기
df_lap = pd.read_csv("./data/laptop.csv")

In [125]:
### tablet 파일 불러오기
df_ta = pd.read_csv("./data/tablet.csv")

In [12]:
### 파일 concat으로 통합
con_df = pd.concat([df_smart, df_lap, df_ta], axis=0, ignore_index=True)
con_df.to_csv("./data/total.csv",index=False)

In [142]:
df = pd.read_csv("./data/total.csv")
df

,ID,point,condition,r_iris_x,r_iris_y,l_iris_x,l_iris_y,r_eyelid_x,r_eyelid_y,l_eyelid_x,l_eyelid_y,l_center_x,l_center_y,r_center_x,r_center_y
0,1,"[1020, 782]",A,987.5,362.0,816.0,386.5,991.6875,365.2500,818.0625,390.5000,816.0,386.5,987.5,362.0
1,1,"[1020, 782]",A,989.0,363.0,818.0,388.0,993.1875,366.8750,818.9375,392.6875,818.0,388.0,989.0,363.0
2,1,"[1020, 782]",A,989.5,362.5,819.0,388.0,993.0000,366.9375,819.5625,392.4375,819.0,388.0,989.5,362.5
3,1,"[1020, 782]",A,988.5,363.0,817.0,388.0,990.5625,366.3750,819.3750,391.2500,817.0,388.0,988.5,363.0
4,1,"[1020, 782]",A,986.0,362.0,814.5,387.0,990.4375,365.3125,815.3125,390.7500,814.5,387.0,986.0,362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232714,30,"[735, 501]",S,603.0,936.0,606.5,636.0,599.8125,946.2500,601.1250,630.5000,606.5,636.0,603.0,936.0
232715,30,"[735, 501]",S,603.5,935.0,609.0,635.5,599.9375,942.3750,601.6250,631.8750,609.0,635.5,603.5,935.0
232716,30,"[735, 501]",S,596.5,943.5,600.5,630.0,593.1250,945.6250,584.8750,629.0625,600.5,630.0,596.5,943.5
232717,30,"[735, 501]",S,602.5,937.0,602.0,637.0,599.4375,945.0000,597.6250,631.6875,602.0,637.0,602.5,937.0


In [143]:
### 데이터 배열처리
data = df.iloc[:, 3:].to_numpy()
target = df.iloc[:, 2].to_numpy()

print(data.shape, target.shape)

(232719, 12) (232719,)


In [144]:
### target 데이터를 숫자로 인코딩
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encoder = LabelEncoder()

# 데이터 변환
target = label_encoder.fit_transform(target)

print(target.shape)

(232719,)


### 데이터 분리하기

In [145]:
### 8:2로 나누기
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(data, target, random_state=42, test_size=0.2)

print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

(186175, 12) (186175,)
(46544, 12) (46544,)


### 정규화

In [146]:
### StandardScaler
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_x)

train_std_scaler = ss.transform(train_x)
test_std_scaler = ss.transform(test_x)

print(train_std_scaler.shape)

(186175, 12)


### 결정트리

In [31]:
from sklearn.tree import DecisionTreeClassifier
# 모델생성
dt = DecisionTreeClassifier(max_depth=42, random_state=42)
# 훈련시키기
dt.fit(train_std_scaler, train_y)
tr_score = dt.score(train_std_scaler, train_y)
ts_score = dt.score(test_std_scaler, test_y)
df_model = []
df_tr = []
df_te = []
df_model.append(dt)
df_tr.append(tr_score)
df_te.append(ts_score)
print("훈련데이터 정확도 :", tr_score,
      "\n테스트데이터 정확도 :", ts_score)

훈련데이터 정확도 : 0.9999843808571786 
테스트데이터 정확도 : 0.836321609295933


In [35]:
df_model = []
df_tr = []
df_te = []
df_model.append(dt)
df_tr.append(tr_score)
df_te.append(ts_score)

In [36]:
### 결정트리 교차검증
from sklearn.tree import DecisionTreeClassifier
params = {"max_depth" : range(5,20,1),
          "min_samples_split" : range(2,100,1),
          "min_impurity_decrease" : np.arange(0.0001, 0.001,0.0001)}
### AutoML에 적용할 훈련모델 생성하기
dt = DecisionTreeClassifier(random_state=42)

### AutoML 클래스 생성하기
gs = GridSearchCV(dt, params, n_jobs=-1)

### AutoML 실행하기 : 하이퍼파라메터 튜닝 + 훈련(교차검증 수행)
gs.fit(train_std_scaler, train_y)

### 정확도 확인하기
tr_score = gs.score(train_std_scaler, train_y)
ts_score = gs.score(test_std_scaler, test_y)
print(gs.best_params_)
print("훈련데이터 정확도 :", tr_score,
      "\n테스트데이터 정확도 :", ts_score)

KeyboardInterrupt: 

### 랜덤포레스트

In [354]:
### 랜덤포레스트 훈련모델 라이브러리
from sklearn.ensemble import RandomForestClassifier

### 훈련모델 생성
rf = RandomForestClassifier(n_jobs=-1,n_estimators=300,min_samples_split=3 ,random_state=42)

### Std훈련 시키기
rf.fit(train_std_scaler, train_y)

### 훈련정확도 및 테스트 정확도
tr_score = rf.score(train_std_scaler, train_y)
ts_score = rf.score(test_std_scaler, test_y)
# df_model.append(rf)
# df_tr.append(tr_score)
# df_te.append(ts_score)
print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

훈련 정확도 : 0.9999687617143571
테스트 정확도 : 0.9155369525832449


In [193]:
sum(y_pred==test_y)/len(test_y)

0.9155369525832449

In [353]:
### 랜덤포레스트 교차검증
from sklearn.ensemble import RandomForestClassifier

params = {"max_depth" : range(10,50,1),
          "n_estimators" : range(10,50,1)}

### 훈련모델 생성
rf = RandomForestClassifier(random_state=42)

### AutoML 클래스 생성하기
gs = GridSearchCV(rf, params, n_jobs=-1)

### AutoML 실행하기 : 하이퍼파라메터 튜닝 + 훈련(교차검증 수행)
gs.fit(train_std_scaler, train_y)

rf = gs.best_estimator_

### 과적합 확인하기
print(gs.best_params_)
print("훈련 정확도 :", rf.score(train_std_scaler, train_y))
print("테스트 정확도 :", rf.score(test_std_scaler, test_y))

KeyboardInterrupt: 

In [ ]:
### 엑스트라트리 교차검증
from sklearn.ensemble import ExtraTreesClassifier

params = {"max_depth" : range(10,100,1),
          "n_estimators" : range(10,200,1)}

### 훈련모델 생성
et = ExtraTreesClassifier(random_state=42)

### AutoML 클래스 생성하기
gs = GridSearchCV(et, params, n_jobs=-1)

### AutoML 실행하기 : 하이퍼파라메터 튜닝 + 훈련(교차검증 수행)
gs.fit(train_std_scaler, train_y)

et = gs.best_estimator_

### 과적합 확인하기
print(gs.best_params_)
print("훈련 정확도 :", et.score(train_std_scaler, train_y))
print("테스트 정확도 :", et.score(test_std_scaler, test_y))

In [ ]:
### HGB 교차검증
from sklearn.ensemble import ExtraTreesClassifier

params = {"n_estimators" : range(10,200,1),
          "max_depth" : range(3,10,1)}

### 훈련모델 생성
hgb = HistGradientBoostingClassifier(random_state=42)

### AutoML 클래스 생성하기
gs = GridSearchCV(hgb, params, n_jobs=-1)

### AutoML 실행하기 : 하이퍼파라메터 튜닝 + 훈련(교차검증 수행)
gs.fit(train_std_scaler, train_y)

hgb = gs.best_estimator_

### 과적합 확인하기
print(gs.best_params_)
print("훈련 정확도 :", hgb.score(train_std_scaler, train_y))
print("테스트 정확도 :", hgb.score(test_std_scaler, test_y))

### 엑스트라트리 모델

In [136]:
from sklearn.ensemble import ExtraTreesClassifier
### 모델 생성
et = ExtraTreesClassifier(n_jobs=-1,max_depth=32,n_estimators=600, random_state=42)

### 훈련시키기
et.fit(train_std_scaler, train_y)

### 정확도 확인
tr_score = et.score(train_std_scaler, train_y)
ts_score = et.score(test_std_scaler, test_y)
# df_model[2] = et
# df_tr[2] = tr_score
# df_te[2] = ts_score
print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

훈련 정확도 : 0.9999677722572848
테스트 정확도 : 0.9150696115503609


In [170]:
df_model

[DecisionTreeClassifier(max_depth=42, random_state=42),
 RandomForestClassifier(n_jobs=-1, random_state=42),
 ExtraTreesClassifier(max_depth=29, n_estimators=276, n_jobs=-1, random_state=42),
 GradientBoostingClassifier(random_state=42),
 HistGradientBoostingClassifier(random_state=42),
 KNeighborsClassifier(n_neighbors=3),
 LogisticRegression(C=30, max_iter=400, n_jobs=-1),
 SGDClassifier(loss='log_loss', random_state=42)]

In [124]:
### 엑스트라 교차검증

params = {"max_depth" : range(1,25,1),
          "n_estimators" : range(2,40,1)}

### 훈련모델 생성
et = ExtraTreesClassifier(random_state=42)

### AutoML 클래스 생성하기
gs = GridSearchCV(et, params, n_jobs=-1)

### AutoML 실행하기 : 하이퍼파라메터 튜닝 + 훈련(교차검증 수행)
gs.fit(train_std_scaler, train_y)

et = gs.best_estimator_

### 과적합 확인하기
print(gs.best_params_)
print("훈련 정확도 :", et.score(train_std_scaler, train_y))
print("테스트 정확도 :", et.score(test_std_scaler, test_y))

KeyboardInterrupt: 

### 그레디언트 부스팅

In [355]:
from sklearn.ensemble import GradientBoostingClassifier
### 모델 생성
gb = GradientBoostingClassifier(random_state=42,n_estimators=100,max_depth=3,learning_rate=0.01)

### 훈련
gb.fit(train_std_scaler, train_y)

### 정확도
tr_score = gb.score(train_std_scaler, train_y)
ts_score = gb.score(test_std_scaler, test_y)
# df_model.append(gb)
# df_tr.append(tr_score)
# df_te.append(ts_score)
print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

KeyboardInterrupt: 

### Histogram-base Gradient Boosting

In [380]:
from sklearn.ensemble import HistGradientBoostingClassifier
### 모델 생성
hgb = HistGradientBoostingClassifier(random_state=42,max_depth=40,learning_rate=0.1,max_iter=3000)

### 훈련
hgb.fit(train_std_scaler, train_y)

### 정확도
tr_score = hgb.score(train_std_scaler, train_y)
ts_score = hgb.score(test_std_scaler, test_y)
df_model.append(hgb)
df_tr.append(tr_score)
df_te.append(ts_score)
print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

훈련 정확도 : 0.9795701611895539
테스트 정확도 : 0.8866745798713063


In [381]:
0.9795701611895539-0.8866745798713063

0.0928955813182476

In [184]:
df_re = pd.DataFrame({'모델':df_model,'훈련정확도':df_tr,'테스트정확도':df_te})
df_re.to_csv("./data/model_score.csv",index=False)

In [182]:
df_model

[DecisionTreeClassifier(max_depth=42, random_state=42),
 RandomForestClassifier(n_jobs=-1, random_state=42),
 ExtraTreesClassifier(max_depth=29, n_estimators=276, n_jobs=-1, random_state=42),
 GradientBoostingClassifier(random_state=42),
 HistGradientBoostingClassifier(random_state=42),
 KNeighborsClassifier(n_neighbors=3),
 LogisticRegression(C=30, max_iter=400, n_jobs=-1),
 SGDClassifier(loss='log_loss', random_state=42)]

In [15]:
df_re

NameError: name 'df_re' is not defined

In [90]:
df_model.append(kn)
df_tr.append(0.943134)
df_te.append(0.886666)

In [122]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=30,max_iter=400,n_jobs=-1)
### 훈련시키기
lr.fit(train_std_scaler, train_y)
### 정확도
tr_score = lr.score(train_std_scaler, train_y)
ts_score = lr.score(test_std_scaler, test_y)
df_model.append(lr)
df_tr.append(tr_score)
df_te.append(ts_score)
print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

훈련 정확도 : 0.26654067224790706
테스트 정확도 : 0.25938651839820076


In [315]:
### 엑스트라 교차검증

params = {"C" : [ 0.001, 0.01, 0.1, 1, 10, 100],
          "max_iter" : range(2,100,1),
          "penalty" : ['l1','l2']}

### 훈련모델 생성
lr = LogisticRegression()

### 훈련시키기
lr.fit(train_std_scaler, train_y)

### AutoML 클래스 생성하기
gs = GridSearchCV(lr, params, n_jobs=-1)

### AutoML 실행하기 : 하이퍼파라메터 튜닝 + 훈련(교차검증 수행)
gs.fit(train_std_scaler, train_y)

et = gs.best_estimator_

### 과적합 확인하기
print(gs.best_params_)
print("훈련 정확도 :", et.score(train_std_scaler, train_y))
print("테스트 정확도 :", et.score(test_std_scaler, test_y))

C:\Users\USER\anaconda3\envs\eyepro\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

C:\Users\USER\anaconda3\envs\eyepro\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning:


2940 fits failed out of a total of 5880.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2940 fits failed with the following error:
Traceba

{'C': 100, 'max_iter': 97, 'penalty': 'l2'}
훈련 정확도 : 0.2660252405347994
테스트 정확도 : 0.25713750234272503


C:\Users\USER\anaconda3\envs\eyepro\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [129]:
from sklearn.linear_model import SGDClassifier

sc = SGDClassifier(loss = "log_loss",max_iter=1000,random_state=42)

sc.fit(train_std_scaler, train_y)

tr_score = sc.score(train_std_scaler, train_y)
ts_score = sc.score(test_std_scaler, test_y)
df_model.append(sc)
df_tr.append(tr_score)
df_te.append(ts_score)
print('훈련 정확도 :', tr_score)
print('테스트 정확도 :', ts_score)

훈련 정확도 : 0.2375671623141322
테스트 정확도 : 0.23377272443306055


In [106]:
del df_model[6]

In [107]:
del df_tr[6]

In [108]:
del df_te[6]

In [213]:
spliter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)
et = ExtraTreesClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)
hgb = HistGradientBoostingClassifier(random_state=42)
kn = KNeighborsClassifier()
lr = LogisticRegression()
sc = SGDClassifier(loss = "log_loss",random_state=42)
scores_list=[]
for models in [dt,rf,et,gb,hgb,kn,lr,sc]:
    scores = cross_validate(models, train_std_scaler, train_y, cv= spliter)
    scores_list.append(scores['test_score'])

C:\Users\USER\anaconda3\envs\eyepro\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USER\anaconda3\envs\eyepro\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

In [65]:
df_scores = pd.read_csv('./data/model_score.csv')
df_scores['모델'] = ['DT','RF','ET','GB','HGB','KNN','LR','SGD']
df_scores = df_scores.drop(index=[3,6,7]).reset_index(drop=True)
df_scores_sort = df_scores.sort_values(by=['훈련정확도','테스트정확도'],ascending=False)
df_scores_sort.to_csv("./data/5model_acc_sort.csv",index=False)

In [33]:
# scores_list = 
arr1 = np.array([0.82289552, 0.82195846, 0.82664376, 0.82664376, 0.82161824,
        0.82286785, 0.82442987, 0.83302093, 0.83458294, 0.81818182])
arr2 = np.array([0.91097923, 0.90957364, 0.91535218, 0.90801187, 0.91283974,
        0.90424867, 0.90893471, 0.91112152, 0.91237113, 0.9084661 ])
arr3 = np.array([0.92893956, 0.92441043, 0.92550367, 0.91972513, 0.92549203,
        0.91986879, 0.92486723, 0.92439863, 0.92908466, 0.921587  ])
arr4 = np.array([0.39856317, 0.38809933, 0.38107137, 0.38809933, 0.37378944,
        0.37425804, 0.37988129, 0.37925648, 0.3947204 , 0.37644486])
arr5 = np.array([0.69982821, 0.68061846, 0.67687022, 0.68311729, 0.67728835,
        0.67307092, 0.67947516, 0.6841612 , 0.69040925, 0.66885348])
arr6 = np.array([0.86834296, 0.8694362 , 0.86693737, 0.86490707, 0.86519838,
        0.86629178, 0.8689472 , 0.86660419, 0.87347704, 0.86504217])
arr7 = np.array([0.2523817 , 0.25175699, 0.25816024, 0.25222552, 0.25445173,
        0.25242112, 0.25866917, 0.25788816, 0.25523274, 0.25476414])
arr8 = np.array([0.2348899 , 0.2334843 , 0.22645635, 0.2503514 , 0.24008122,
        0.22258669, 0.22118088, 0.24023743, 0.22914714, 0.23867541])
scores_list = [arr1,arr2,arr3,arr4,arr5]

In [73]:
np.savetxt('./data/5model_10fold_acc.csv',scores_list)


In [295]:
ranks[0] = [0,0,0,0,0,0,0,0]
ranks[1] = [0,0,0,0,0,0,0,0]
ranks[2] = [0,0,0,0,0,0,0,0]
ranks[3] = [4,4,2,3,1,3,1,2]
ranks[4] = [4,4,2,3,1,3,1,2]
ranks[5] = [4,4,2,3,1,3,1,2]
ranks[6] = [4,4,2,3,1,3,1,2]
ranks

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [4, 4, 2, 3, 1, 3, 1, 2],
       [4, 4, 2, 3, 1, 3, 1, 2],
       [4, 4, 2, 3, 1, 3, 1, 2],
       [4, 4, 2, 3, 1, 3, 1, 2],
       [8, 7, 4, 5, 1, 6, 2, 3],
       [8, 7, 4, 5, 1, 6, 2, 3],
       [8, 7, 4, 5, 1, 6, 2, 3]], dtype=int64)

In [320]:
# for i in range(8):
#     scores_list[i] = np.delete(scores_list[i],10)
scores_list
ranks = []
for i in range(10): # fold 수 만큼 반복
    rank = np.argsort([s[i] for s in scores_list]) +1 # 작은 값부터 큰 값까지 오름차순으로 인덱스 반환 후 +1 (순위는 일반적으로 작은 값부터 시작)
    ranks.append(rank)
ranks = np.array(ranks)
ranks
scores_list


array([8, 7, 4, 5, 1, 6, 2, 3], dtype=int64)

In [346]:
# np.delete(scores_list[0],10)
np.argsort([-arr[0] for arr in scores_list])+1
[arr[0] for arr in scores_list]
scores_list

[array([0.82289552, 0.82195846, 0.82664376, 0.82664376, 0.82161824,
        0.82286785, 0.82442987, 0.83302093, 0.83458294, 0.81818182]),
 array([0.91097923, 0.90957364, 0.91535218, 0.90801187, 0.91283974,
        0.90424867, 0.90893471, 0.91112152, 0.91237113, 0.9084661 ]),
 array([0.92893956, 0.92441043, 0.92550367, 0.91972513, 0.92549203,
        0.91986879, 0.92486723, 0.92439863, 0.92908466, 0.921587  ]),
 array([0.39856317, 0.38809933, 0.38107137, 0.38809933, 0.37378944,
        0.37425804, 0.37988129, 0.37925648, 0.3947204 , 0.37644486]),
 array([0.69982821, 0.68061846, 0.67687022, 0.68311729, 0.67728835,
        0.67307092, 0.67947516, 0.6841612 , 0.69040925, 0.66885348]),
 array([0.86834296, 0.8694362 , 0.86693737, 0.86490707, 0.86519838,
        0.86629178, 0.8689472 , 0.86660419, 0.87347704, 0.86504217]),
 array([0.2523817 , 0.25175699, 0.25816024, 0.25222552, 0.25445173,
        0.25242112, 0.25866917, 0.25788816, 0.25523274, 0.25476414]),
 array([0.2348899 , 0.2334843 , 0.

In [200]:
DT	RF	ET	GB	HGB	KNN	LR	SGD

,ID,point,condition,r_iris_x,r_iris_y,l_iris_x,l_iris_y,r_eyelid_x,r_eyelid_y,l_eyelid_x,l_eyelid_y,l_center_x,l_center_y,r_center_x,r_center_y
0,1,"[1020, 782]",A,987.5,362.0,816.0,386.5,991.6875,365.2500,818.0625,390.5000,816.0,386.5,987.5,362.0
1,1,"[1020, 782]",A,989.0,363.0,818.0,388.0,993.1875,366.8750,818.9375,392.6875,818.0,388.0,989.0,363.0
2,1,"[1020, 782]",A,989.5,362.5,819.0,388.0,993.0000,366.9375,819.5625,392.4375,819.0,388.0,989.5,362.5
3,1,"[1020, 782]",A,988.5,363.0,817.0,388.0,990.5625,366.3750,819.3750,391.2500,817.0,388.0,988.5,363.0
4,1,"[1020, 782]",A,986.0,362.0,814.5,387.0,990.4375,365.3125,815.3125,390.7500,814.5,387.0,986.0,362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80026,29,"[500, 803]",S,743.0,767.5,465.5,754.5,742.1250,770.9375,464.5000,758.3125,465.5,754.5,743.0,767.5
80027,29,"[500, 803]",S,747.5,767.0,468.5,752.0,743.8750,769.9375,464.0000,755.5625,468.5,752.0,747.5,767.0
80028,29,"[500, 803]",S,746.0,767.0,468.5,753.0,744.6875,769.8750,464.4375,756.4375,468.5,753.0,746.0,767.0
80029,29,"[500, 803]",S,745.5,767.0,467.5,752.5,744.3125,770.1250,464.2500,756.1250,467.5,752.5,745.5,767.0


In [154]:
### 라이브러리
# - 정확도
from sklearn.metrics import accuracy_score
# - 정밀도
from sklearn.metrics import precision_score
# - 재현율
from sklearn.metrics import recall_score
# - f1-score
from sklearn.metrics import f1_score

In [190]:
### 정확도
acc = accuracy_score(test_y,pred)
### 정밀도
pre = precision_score(test_y,pred, average='macro')
### 재현율
rec = recall_score(test_y,pred, average='macro')
### f1-score
f1 = f1_score(test_y,pred, average='macro')
acc,pre,rec,f1

(0.9150696115503609,
 0.9164005824799301,
 0.9071366547543469,
 0.9107720793168372)

## 이미지 인식

In [ ]:
import cv2

face_cascade = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./data/haarcascade_eye.xml')

img = cv2.imread('./data/1model.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3,5)
for (x,y,w,h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray,1.1,5)
    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh),(0,255,0),2)

# cv2.imshow('img',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
plt.imshow(img)

In [ ]:
img = cv2.imread('./data/1model.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(gray)

In [ ]:
# XML 파일 경로
xml_file_path = '../haarcascades/haarcascade_frontalface_default.xml'

# 파일 존재 여부 확인
if os.path.exists(xml_file_path):
    print("XML 파일이 존재합니다.")
else:
    print("XML 파일이 존재하지 않습니다.")

In [ ]:
pd.read_csv("./data/smartphone_label.csv")

In [ ]:
import cv2
import numpy as np
import json

# JSON 파일에서 정보를 불러오기
with open('./data/data.json', 'r') as f:
    data = json.load(f)
annotations = data['Annotations']['annotations']

img = cv2.imread('./data/1model.png')

condition = data['Annotations']['condition']
inst = data['Annotations']['inst']
posture = data['Annotations']['posture']

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_thickness = 2
text_color = (0, 0, 255)

cv2.putText(img, f"Condition: {condition}", (10, 30), font, font_scale, text_color, font_thickness)
cv2.putText(img, f"Inst: {inst}", (10, 60), font, font_scale, text_color, font_thickness)
cv2.putText(img, f"Posture: {posture}", (10, 90), font, font_scale, text_color, font_thickness)

# 주어진 JSON 어노테이션을 화면에 표시
for annotation in annotations:
    points = annotation['points']

    if annotation['shape'] == 'Ellipse':
        rx = int(annotation['rx'])
        ry = int(annotation['ry'])
        cx, cy = int(annotation['cx']), int(annotation['cy'])
        thick = 2
        color = (0, 255, 0)
        cv2.ellipse(img, (cx, cy), (rx, ry), 0, 0, 360, color, thick)

    elif annotation['shape'] == 'Polygon':
        contours = np.array(points, np.int32)
        contours = contours.reshape(-1, 1, 2)
        color = (0, 0, 255)
        cv2.polylines(img, [contours], isClosed=True, color=color, thickness=2)

    elif annotation['shape'] == 'Point':
        x, y = int(points[0][0]), int(points[0][1])
        radius = 3
        color = (255, 0, 0)
        cv2.circle(img, (x, y), radius, color, -1)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import dlib
import numpy as np

def detect_eyes(gray_img, face_landmarks):
    eye_regions = []

    for side in ['left', 'right']:
        if side == 'left':
            p1, p2, p3, p4, p5, p6 = list(range(36, 42))
        else:
            p1, p2, p3, p4, p5, p6 = list(range(42, 48))

        pt1, pt2, pt3, pt4, pt5, pt6 = [face_landmarks.part(p1).x, face_landmarks.part(p1).y], [face_landmarks.part(p2).x, face_landmarks.part(p2).y], [face_landmarks.part(p3).x, face_landmarks.part(p3).y], [face_landmarks.part(p4).x, face_landmarks.part(p4).y], [face_landmarks.part(p5).x, face_landmarks.part(p5).y], [face_landmarks.part(p6).x, face_landmarks.part(p6).y]

        eye_region = np.array([pt1, pt2, pt3, pt4, pt5, pt6], np.int32)
        eye_regions.append(eye_region)

    return eye_regions

def detect_iris(eye_image):
    gray_img = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.medianBlur(gray_img, 5)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 20, param1=30, param2=15, minRadius=7, maxRadius=21)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cv2.circle(eye_image, (i[0], i[1]), i[2], (0, 255, 0), 2)
            
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("./data/shape_predictor_68_face_landmarks.dat")

img = cv2.imread("./data/img.jpg")
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_detector(gray_img)

for face in faces:
    landmarks = landmark_predictor(gray_img, face)
    eye_regions = detect_eyes(gray_img, landmarks)

    for points in eye_regions:
        eye_image = img[points[1][1]:points[5][1], points[0][0]:points[3][0]]
        detect_iris(eye_image)
        for point in points:
            x,y=point
            print(f"좌표: ({x}, {y})")
        points = points.reshape((-1, 1, 2))
        cv2.polylines(img, [points], isClosed=True, color=(0, 0, 255), thickness=2)

cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import dlib
import numpy as np

def detect_eyes(gray_img, face_landmarks):
    eye_regions = []

    for side in ['left', 'right']:
        if side == 'left':
            p1, p2, p3, p4, p5, p6 = list(range(36, 42))
        else:
            p1, p2, p3, p4, p5, p6 = list(range(42, 48))

        pt1, pt2, pt3, pt4, pt5, pt6 = [face_landmarks.part(p1).x, face_landmarks.part(p1).y], [face_landmarks.part(p2).x, face_landmarks.part(p2).y], [face_landmarks.part(p3).x, face_landmarks.part(p3).y], [face_landmarks.part(p4).x, face_landmarks.part(p4).y], [face_landmarks.part(p5).x, face_landmarks.part(p5).y], [face_landmarks.part(p6).x, face_landmarks.part(p6).y]

        eye_region = np.array([pt1, pt2, pt3, pt4, pt5, pt6], np.int32)
        eye_regions.append(eye_region)

    return eye_regions

def detect_pupil(img, eye_points):
    eye_img = img[eye_points[1][1]:eye_points[5][1], eye_points[0][0]:eye_points[3][0]]
    gray_eye = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
    blurred_eye = cv2.GaussianBlur(gray_eye, (7, 7), 0)

    circles = cv2.HoughCircles(blurred_eye, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=3, maxRadius=8)

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            x += eye_points[0][0]
            y += eye_points[1][1]
            cv2.circle(eye_img, (x - eye_points[0][0], y - eye_points[1][1]), r, (0, 255, 0), 2)
            # 눈동자의 중심 좌표에 원 그리기
            cv2.circle(eye_img, (x - eye_points[0][0], y - eye_points[1][1]), 2, (0, 255, 0), -1)

    # 눈동자 표시된 이미지를 원래 이미지에 적용
    img[eye_points[1][1]:eye_points[5][1], eye_points[0][0]:eye_points[3][0]] = eye_img

face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("./data/shape_predictor_68_face_landmarks.dat")

img = cv2.imread("./data/img.jpg")
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_detector(gray_img)

for face in faces:
    landmarks = landmark_predictor(gray_img, face)
    eye_regions = detect_eyes(gray_img, landmarks)

    for eye_points in eye_regions:
        detect_pupil(img, eye_points)
        points = eye_points.reshape((-1, 1, 2))
        cv2.polylines(img, [points], isClosed=True, color=(0, 0, 255), thickness=2)

cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import dlib
import cv2

# EYES = list(range(36,48))
EYES = list(range(36, 42)) + list(range(42, 48))
frame_width = 640
frame_height = 480

raw_image = cv2.imread('./data/1model.png') #-- 이미지 경로

face_cascade_name = './data/haarcascade_frontalface_alt.xml'
face_cascade = cv2.CascadeClassifier()
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)

predictor_file = './data/shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(predictor_file)

image = cv2.resize(raw_image, (frame_width, frame_height))
faces = face_cascade.detectMultiScale(image)

#- detect face area
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
        
    #- detect eye area
    points = np.matrix([[p.x, p.y] for p in predictor(image, rect).parts()])
    show_parts = points[EYES]

    for (i, point) in enumerate(show_parts):
        x = point[0,0]
        y = point[0,1]
        cv2.circle(image, (x, y), 1, (0, 255, 255), -1)

cv2.imshow('Face & Eye Area Detection', image)
#cv2.imwrite('output.png',image) #- image output 저장용
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import dlib
import cv2

EYES = list(range(36,48))

frame_width = 640
frame_height = 480

face_cascade_name = './data/haarcascade_frontalface_alt.xml' #-- 본인 환경에 맞게 변경할 것
face_cascade = cv2.CascadeClassifier()
vedio_path = './vedio.mp4' #-- 본인 환경에 맞게 변경할 것


if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)

predictor_file = './data/shape_predictor_68_face_landmarks.dat' #-- 본인 환경에 맞게 변경할 것
predictor = dlib.shape_predictor(predictor_file)

cap = cv2.VideoCapture(0) #-- 캠 사용 시 vedio_path 대신 0 사용    

if not cap.isOpened:
    print('Could not open video')
    exit(0)

"""
#- vedio output 저장용
w = round(frame_width) # round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(frame_height) # round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 20 # cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('output_sample.mp4', fourcc, fps, (w, h))

"""

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print('Could not read frame')
        exit()

    image = cv2.resize(frame, (frame_width, frame_height))
    faces = face_cascade.detectMultiScale(image)
    
    #- detect face area
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
        
        #- detect eye area
        points = np.matrix([[p.x, p.y] for p in predictor(image, rect).parts()])
        show_parts = points[EYES]

        for (i, point) in enumerate(show_parts):
            x = point[0,0]
            y = point[0,1]
            cv2.circle(image, (x, y), 1, (0, 255, 255), -1)

    #out.write(image) #- vedio output 저장용
    cv2.imshow('Face & Eye Area Detection', image)
    
    # q 입력시 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from l2cs.model import L2CS
from l2cs.utils import draw_gaze

# Load pre-trained L2CSNet model
model = L2CS.load_model("./data/data.json")

# Load image
image = cv2.imread("./data/1model.png")

# Perform eye tracking using L2CSNet
eye_coordinates = model.predict(image)

# Draw eye coordinates on the image
image_with_eyes = draw_eyes(image, eye_coordinates)

# Display the image with eye tracking
cv2.imshow("Eye Tracking", image_with_eyes)
cv2.waitKey(0)
cv2.destroyAllWindows()